In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
match_file = "/content/gdrive/My Drive/ML_project/Matches_clean.csv"
ball_file ="/content/gdrive/My Drive/ML_project/ball_filtered.csv"
raw_match_file = "/content/gdrive/My Drive/ML_project/IPLMatches2008-2020.csv"

ball = pd.read_csv(ball_file)
matches = pd.read_csv(raw_match_file)
clean_matches = pd.read_csv(match_file)

ball.shape

(193468, 18)

# Append year column

In [ ]:
matches['yr'] = matches['date'].apply(lambda d: int(d.split('-')[0]))
match_dfs={}
for yr in range(2008, 2021):
  match_dfs[yr] = matches.loc[matches['yr'] == yr]

id_yr = {}
for index, row in matches.iterrows():
  id_yr[row['id']] = row['yr']

# Calc Points - by Jahnvi


In [ ]:
def retpos(ids):
  points={}
  lst=['bowled','run out','lbw','stumped','caught','caught and bowled','hit wicket']
  for index, row in ball.iterrows():
    if row['id'] in ids:
      if row['batsman_runs']==4:
        try:
          points[row['batsman']]+=2.5
        except:
          points[row['batsman']]=2.5
      
      elif row['batsman_runs']==6:
        try:
          points[row['batsman']]+=3.5
        except:
          points[row['batsman']]=3.5

      if row['total_runs']==0:
        try:
          points[row['bowler']]+=1
        except:
          points[row['bowler']]=1

      if row['dismissal_kind']=='caught' or row['dismissal_kind']=='stumped':
        try:
          points[row['fielder']]+=2.5
        except:
          points[row['fielder']]=2.5
      
      if row['dismissal_kind'] in lst:
        try:
          points[row['bowler']]+=3.5
        except:
          points[row['bowler']]=3.5
  return points    

In [ ]:
points={}
for yr in match_dfs.keys():
  id_lst = list(match_dfs[yr]['id'])
  points[yr] = retpos(id_lst)
  print(points[yr])

{'Z Khan': 161.5, 'JH Kallis': 148.0, 'SB Joshi': 21.5, 'BB McCullum': 87.5, 'RT Ponting': 13.5, 'P Kumar': 202.0, 'DJ Hussey': 154.0, 'AA Noffke': 12.0, 'CL White': 41.5, 'Mohammad Hafeez': 56.0, 'SC Ganguly': 218.0, 'AB Agarkar': 113.5, 'MV Boucher': 91.0, 'M Kartik': 66.0, 'WP Saha': 79.5, 'LR Shukla': 104.0, 'I Sharma': 142.0, 'AB Dinda': 157.0, 'K Goel': 38.5, 'JDP Oram': 64.0, 'MEK Hussey': 73.5, 'JR Hopes': 177.0, 'MS Gony': 235.0, 'PP Chawla': 224.5, 'SK Raina': 186.5, 'WA Mota': 16.5, 'IK Pathan': 273.5, 'S Badrinath': 93.0, 'B Lee': 72.5, 'ML Hayden': 81.0, 'PA Patel': 130.5, 'S Sreesanth': 203.5, 'KC Sangakkara': 140.5, 'M Muralitharan': 175.5, 'P Amarnath': 58.5, 'Joginder Sharma': 90.5, 'Yuvraj Singh': 166.5, 'SM Katich': 34.5, 'GD McGrath': 209.5, 'T Kohli': 6.0, 'B Geeves': 29.5, 'YK Pathan': 296.5, 'SK Warne': 209.0, 'G Gambhir': 203.0, 'SR Watson': 414.5, 'S Dhawan': 135.5, 'SK Trivedi': 159.5, 'DL Vettori': 23.0, 'MF Maharoof': 207.5, 'D Salunkhe': 34.0, 'MM Patel': 2

In [ ]:
player_dict={}

for yr in points:
  for player in points[yr].keys():
    try:
      player_dict[player][yr]=points[yr][player]
    except:
      player_dict[player]={}
      player_dict[player][yr]=points[yr][player]

In [ ]:
player_dict

{'A Ashish Reddy': {2012: 104.5, 2013: 73.0, 2015: 50.5, 2016: 30.0},
 'A Chandila': {2012: 59.0, 2013: 87.5},
 'A Chopra': {2008: 17.5, 2009: 5.0},
 'A Choudhary': {2017: 65.5},
 'A Dananjaya': {2018: 6},
 'A Flintoff': {2009: 54.0},
 'A Kumble': {2008: 123.0, 2009: 227.5, 2010: 225.0},
 'A Mishra': {2008: 88.0,
  2009: 164.0,
  2010: 213.5,
  2011: 225.0,
  2012: 139.5,
  2013: 260.0,
  2014: 94.0,
  2015: 111.5,
  2016: 147.0,
  2017: 132.5,
  2018: 118.5,
  2019: 128.0,
  2020: 30.0},
 'A Mithun': {2009: 7, 2010: 8.0, 2011: 116.5, 2012: 5, 2013: 12.5},
 'A Mukund': {2008: 7.5, 2013: 2.5},
 'A Nehra': {2008: 186.5,
  2009: 221.0,
  2010: 71.5,
  2012: 146.0,
  2013: 129.5,
  2014: 58.0,
  2015: 265.0,
  2016: 113.0,
  2017: 84.5},
 'A Nel': {2008: 10.5},
 'A Nortje': {2020: 239.5},
 'A Singh': {2009: 87.5, 2011: 110.0, 2012: 102.5},
 'A Symonds': {2008: 85.5, 2009: 175.0, 2010: 329.5, 2011: 59.0},
 'A Uniyal': {2010: 18.5},
 'A Zampa': {2016: 74.0, 2017: 65.0, 2020: 29.5},
 'AA Bila

# Find Team Players


In [ ]:
team_players = {}
for index, row in ball.iterrows():
  if row['batting_team'] not in team_players.keys():
    team_players[row['batting_team']] = {}
    for yr in range (2008, 2021):
      team_players[row['batting_team']][yr] = []
    
  if row['bowling_team'] not in team_players.keys():
    team_players[row['bowling_team']] = {}
    for yr in range (2008, 2021):
      team_players[row['bowling_team']][yr] = []

  team_players[row['batting_team']][id_yr[row['id']]].append(row['batsman'])
  team_players[row['batting_team']][id_yr[row['id']]].append(row['non_striker'])
  team_players[row['bowling_team']][id_yr[row['id']]].append(row['bowler'])

for team in team_players.keys():
  for yr in team_players[team].keys():
    team_players[team][yr] = set(team_players[team][yr])



In [ ]:
points

{2008: {'A Chopra': 17.5,
  'A Kumble': 123.0,
  'A Mishra': 88.0,
  'A Mukund': 7.5,
  'A Nehra': 186.5,
  'A Nel': 10.5,
  'A Symonds': 85.5,
  'AA Noffke': 12.0,
  'AB Agarkar': 113.5,
  'AB Dinda': 157.0,
  'AB de Villiers': 23.5,
  'AB de Villiers (sub)': 5.0,
  'AC Gilchrist': 211.5,
  'AD Mascarenhas': 18.5,
  'AM Nayar': 99.0,
  'AM Rahane': 2.5,
  'AM Rahane (sub)': 2.5,
  'AS Yadav': 22.0,
  'Abdur Razzak': 5,
  'B Akhil': 72.5,
  'B Chipli': 10.0,
  'B Chipli (sub)': 2.5,
  'B Geeves': 29.5,
  'B Lee': 72.5,
  'BAW Mendis': 12.5,
  'BB McCullum': 87.5,
  'BJ Hodge': 7.5,
  'CK Kapugedera': 10.0,
  'CL White': 41.5,
  'CRD Fernando': 93.5,
  'D Kalyankrishna': 19.5,
  'D Salunkhe': 34.0,
  'DB Das': 40.0,
  'DB Ravi Teja': 54.0,
  'DJ Bravo': 177.0,
  'DJ Hussey': 154.0,
  'DJ Thornely': 46.5,
  'DL Vettori': 23.0,
  'DNT Zoysa': 33.5,
  'DP Vijaykumar': 77.0,
  'DPMD Jayawardene': 69.5,
  'DR Smith': 63.5,
  'DS Kulkarni': 127.5,
  'DS Lehmann': 7.5,
  'DT Patil': 7.5,
  'DW

# Clean Data


In [ ]:
clean_matches

,Unnamed: 0,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,umpire1,umpire2
0,0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,Asad Rauf,RE Koertzen
1,1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,MR Benson,SL Shastri
2,2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,Aleem Dar,GA Pratapkumar
3,3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,SJ Davis,DJ Harper
4,4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,BF Bowden,K Hariharan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,810,1216546,Dubai,2020-10-20,S Dhawan,Dubai International Cricket Stadium,0,Delhi Capitals,Kings XI Punjab,Delhi Capitals,bat,Kings XI Punjab,wickets,5.0,C Shamshuddin,RK Illingworth
776,812,1237177,Dubai,2020-11-05,JJ Bumrah,Dubai International Cricket Stadium,0,Mumbai Indians,Delhi Capitals,Delhi Capitals,field,Mumbai Indians,runs,57.0,CB Gaffaney,Nitin Menon
777,813,1237178,Abu Dhabi,2020-11-06,KS Williamson,Sheikh Zayed Stadium,0,Royal Challengers Bangalore,Sunrisers Hyderabad,Sunrisers Hyderabad,field,Sunrisers Hyderabad,wickets,6.0,PR Reiffel,S Ravi
778,814,1237180,Abu Dhabi,2020-11-08,MP Stoinis,Sheikh Zayed Stadium,0,Delhi Capitals,Sunrisers Hyderabad,Delhi Capitals,bat,Delhi Capitals,runs,17.0,PR Reiffel,S Ravi


In [ ]:
clean_matches['yr'] = clean_matches['date'].apply(lambda d: int(d.split('-')[0]))
match_dfs={}
for yr in range(2008, 2020):
  match_dfs[yr] = clean_matches.loc[clean_matches['yr'] == str(yr)]
clean_matches = clean_matches.drop(['Unnamed: 0', 'date'], axis=1)
clean_matches.columns

Index(['id', 'city', 'player_of_match', 'venue', 'neutral_venue', 'team1',
       'team2', 'toss_winner', 'toss_decision', 'winner', 'result',
       'result_margin', 'umpire1', 'umpire2', 'yr'],
      dtype='object')

# Year-wise points of teams

In [ ]:
team_year_points = {}
for team in team_players.keys():
  team_year_points[team] = {}
  for yr in team_players[team].keys():
    total_points = 0
    for player in team_players[team][yr]:
      try:
        total_points += player_dict[player][yr]
      except KeyError:
        total_points += 0
    total = 1 if len(team_players[team][yr]) == 0 else len(team_players[team][yr])
    team_year_points[team][yr] = float(total_points)/ total
team_year_points

{'Chennai Super Kings': {2008: 106.94736842105263,
  2009: 99.97222222222223,
  2010: 91.67391304347827,
  2011: 115.02941176470588,
  2012: 129.26470588235293,
  2013: 136.73529411764707,
  2014: 114.66666666666667,
  2015: 165.64285714285714,
  2016: 0.0,
  2017: 0.0,
  2018: 111.275,
  2019: 114.8157894736842,
  2020: 82.175},
 'Deccan Chargers': {2008: 86.9,
  2009: 110.21052631578948,
  2010: 95.45238095238095,
  2011: 97.61111111111111,
  2012: 77.1086956521739,
  2013: 0.0,
  2014: 0.0,
  2015: 0.0,
  2016: 0.0,
  2017: 0.0,
  2018: 0.0,
  2019: 0.0,
  2020: 0.0},
 'Delhi Capitals': {2008: 0.0,
  2009: 0.0,
  2010: 0.0,
  2011: 0.0,
  2012: 0.0,
  2013: 0.0,
  2014: 0.0,
  2015: 0.0,
  2016: 0.0,
  2017: 0.0,
  2018: 0.0,
  2019: 107.625,
  2020: 104.8},
 'Delhi Daredevils': {2008: 105.67647058823529,
  2009: 98.02631578947368,
  2010: 80.27272727272727,
  2011: 73.79545454545455,
  2012: 99.56818181818181,
  2013: 80.22727272727273,
  2014: 73.275,
  2015: 85.3,
  2016: 88.1666

# Append team points in data

In [ ]:
clean_matches['team1_points']=0.0
clean_matches['team2_points']=0.0

for i in range(clean_matches.shape[0]):
  clean_matches.at[i, 'team1_points'] =  float(team_year_points[clean_matches.iloc[i]['team1']][clean_matches.iloc[i]['yr']])
  clean_matches.at[i, 'team2_points'] =  float(team_year_points[clean_matches.iloc[i]['team2']][clean_matches.iloc[i]['yr']])

In [ ]:
clean_matches.to_csv('matches_with_average_points.csv')

# Team Player Data


In [ ]:
team_player_df = pd.DataFrame(team_players)

In [ ]:
team_player_df.to_csv('team_player_df.csv')

In [ ]:
player_points = pd.DataFrame(player_dict)
player_points.to_csv('player_points.csv')